Responsabilidades:
- Simular la respuesta al escalón de Gcl(s) con los valores hallados.
- Implementar el PID en el script de TempLABUdeA.
- Realizar la prueba práctica, recolectar los datos de temperatura y control.
- Comparar los datos experimentales con la simulación (gráficas, %OS, Ts).

Entregables:
- Código de simulación y control.
- Archivos CSV de los datos experimentales.
- Gráficas comparativas: simulación vs experimento.

In [ ]:
from sympy import symbols, Eq, exp, sqrt, pi, solve, N

zeta, wn, Ts, OS = symbols('zeta wn Ts OS', real=True, positive=True)

OS_value = 10  # 10%
Ts_value = 2   # 2 minutos

# Ecuación del sobreimpulso (OS ≈ 100 * exp(-pi*zeta / sqrt(1 - zeta**2)))
eq1 = Eq(OS, 100 * exp(-pi * zeta / sqrt(1 - zeta**2)))
# Ecuación del tiempo de asentamiento (Ts ≈ 4 / (zeta * wn))
eq2 = Eq(Ts, 4 / (zeta * wn))
# Sustituir valores numéricos
eq1_num = eq1.subs(OS, OS_value)
eq2_num = eq2.subs(Ts, Ts_value)

# Resolver la ecuación de OS para zeta
zeta_solutions = solve(eq1_num, zeta)
# Evaluar las soluciones numéricamente  
zeta_numeric = [N(sol) for sol in zeta_solutions if sol.is_real and sol > 0]
# Usar el valor numérico de zeta para encontrar wn


wn_val = solve(eq2_num.subs(zeta, zeta_numeric[0]), wn)[0]

print("Valor de zeta:", zeta_soc[0])
print("Valor de wn:", wn_val)

Valor de zeta: log(10)/sqrt(log(10)**2 + pi**2)
Valor de wn: 3.38320725639016
